<center>
    <h1> Scientific Programming in Python  </h1>
    <h2> Topic 3: Handling Very Large Arrays, Memory Mappings </h2> 
</center>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

def image_plot(img):
    """
    img is an (nx,ny,3) numpy.array
    """
    plt.figure(figsize=(12,12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

Para esta actividad trabajaremos con la siguiente imagen astronómica:

In [ ]:
img = plt.imread('heic1608b.jpg', format='jpeg')[0:8660,:,:]
rows, columns, channel = img.shape
print(img.shape)

In [ ]:
image_plot(img)

#### Paso 1.

1. Guarde `img` en un archivo `hdf5` en el formato que considere conveniente (puede utilizar _chunks_). Explique su elección. 
2. Elimine `img` de memoria principal.

#### Paso 2

1. Aplíque el siguiente filtro/kernel _sin overlap_ a cada canal de la imágen por separado:
```Python
K = 1/25. * np.ones((5,5))
```
2. No puede cargar `img` completamente en memoria. 
3. Realice `timeit` y `memit` del computo anterior.
4. Guarde la imágen resultante. ¿Que fue lo que se le hizo a la imágen original?

#### Paso 3

Muestre la imágen resultante con la función `image_plot()`.

## Desarrollo

In [ ]:
%load_ext memory_profiler

#### Paso 1 

In [ ]:
try:
    fdata = h5py.File("image.h5", "w")
except IOError:    
    fdata.close()
    fdata = h5py.File("image.h5", "w")

Uso chunks de 100 filas, porque que recorro la imagen por columnas y para que vuela a acceder al disco cada 20  iteraciones de filas al aplicar el kernel.

In [ ]:
imgdset = fdata.create_dataset("RGB", (8660, 10260, 3), chunks=(100,10260,3),dtype='uint8')

In [ ]:
for row in range(rows):
    tmp = img[row,:,:]
    imgdset[row,:] = tmp
    del tmp

In [ ]:
del img

#### Paso 2

In [ ]:
# Kernel
K = 1/25. * np.ones((5,5))
K = K.ravel()
nrows = rows//5
ncolumns = columns//5


out = np.empty((nrows, ncolumns, channel), dtype='uint8')

In [ ]:
def conv(imgdset, K,out):
    for row in range(nrows):
        for col in range(ncolumns):
            r,c=row*5,col*5
            section1 = imgdset[r:r+5,c:c+5].ravel(order="F")
            out[row,col,0]=np.dot(section1[0:25], K)
            out[row,col,1]=np.dot(section1[25:50], K)
            out[row,col,2]=np.dot(section1[50:75], K)
            del section1

In [ ]:
conv(imgdset, K, out)

In [ ]:
%memit conv(imgdset, K, out)

In [ ]:
%timeit conv(imgdset, K, out)

El Kernel aplicado a la imagen reduce el tamaño de la imagen cambiando el número de píxeles que contiene a $\frac{1}{25}$ del original.

#### Paso 3

In [ ]:
image_plot(out)